# **Email Spam Classification-Random Forest**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/email-spam-classification-dataset-csv/emails.csv")
df.head(10)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
X = df.iloc[:,1:3001]
y = df.iloc[:,-1].values
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state=5)

# **Random Forest Classifier**

In [ ]:
rfc = RandomForestClassifier(random_state=4)

rfc.fit(X_train,y_train)

print("Train Results \n")
y_train_pred  = rfc.predict(X_train)
y_train_prob = rfc.predict_proba(X_train)[:,1]

print("Confusion Matrix for Train : \n", confusion_matrix(y_train, y_train_pred))
print("Accuracy Score for Train : ", accuracy_score(y_train, y_train_pred))
print("ROC AUC for Train : ", roc_auc_score(y_train, y_train_prob))

print("+"*50)
print("Test Results \n")
y_test_pred  = rfc.predict(X_test)
y_test_prob = rfc.predict_proba(X_test)[:,1]

print("Confusion Matrix for Test : \n", confusion_matrix(y_test, y_test_pred))
print("Accuracy Score for Test : ", accuracy_score(y_test, y_test_pred))
print("ROC AUC for Test : ", roc_auc_score(y_test, y_test_prob))

**Observations:**
- As the **Accuracy Score** and **ROC_AUC Score** is 1.0.
- It is clear that, our model is overfitting means, it is learning too much from the data.
- Solution: We will RandomizedSearchCV (Cross-Validation Method) to search for best parameters of Random Forest for better prediction.


# **RandomizedSearchCV**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

rfc = RandomForestClassifier(random_state=4)



params = {'n_estimators': sp_randint(50,400),
          'max_features' : sp_randint(2,16),
          'max_depth' : sp_randint(2,10),
          'min_samples_split' : sp_randint(2,25),
          'min_samples_leaf' : sp_randint(1,25),
          'criterion':['gini','entropy']}

rsearch = RandomizedSearchCV(rfc,
                             param_distributions=params,
                             n_iter=50,
                             cv=3, 
                             return_train_score = True,
                             scoring='roc_auc',
                             n_jobs=-1,
                            random_state=5)

rsearch.fit(X,y)

In [ ]:
rsearch.best_params_

# **Applying Best Params**

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

rfc = RandomForestClassifier(**rsearch.best_params_,random_state=4,oob_score=True)
rfc.fit(X_train,y_train)

print("Train Results \n")
y_train_pred  = rfc.predict(X_train)
y_train_prob = rfc.predict_proba(X_train)[:,1]

print("Confusion Matrix for Train : \n", confusion_matrix(y_train, y_train_pred))
print("Accuracy Score for Train : ", accuracy_score(y_train, y_train_pred))
print("ROC AUC for Train : ", roc_auc_score(y_train, y_train_prob))
print("OOB Score for Train : ", rfc.oob_score_)

print("+"*50)
print("Test Results \n")
y_test_pred  = rfc.predict(X_test)
y_test_prob = rfc.predict_proba(X_test)[:,1]

print("Confusion Matrix for Test : \n", confusion_matrix(y_test, y_test_pred))
print("Accuracy Score for Test : ", accuracy_score(y_test, y_test_pred))
print("ROC AUC for Test : ", roc_auc_score(y_test, y_test_prob))

# **ROC-AUC Curve for Test Data**

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test,y_test_prob)
thresholds[0] = thresholds[0]-1

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
ax.plot(fpr,tpr,'black')
ax.plot(fpr,fpr,'green')
ax1=ax.twinx()
ax1.plot(fpr,thresholds)
ax1.set_ylabel("Thresholds")
ax.set_xlabel("FPR")
ax.set_ylabel("TPR")
plt.show()